# Ultimate RVC Maker 🎵

An high-quality voice conversion tool focused on experimenting and performance, based on Vietnamese-RVC (which is an RVC Fork).

_____________________________________________________________
[Support](https://discord.gg/aihub) | [GitHub](https://github.com/unchCrew/RVC-MAKER.git) | [Terms of Use](https://github.com/unchCrew/RVC-MAKER/blob/main/TERMS_OF_USE.md)
_____________________________________________________________

**This project was created by [TheNeoDev](https://github.com/TheNeodev)** based on [PhamHuynhAnh16/Vietnamese-RVC](https://github.com/PhamHuynhAnh16/Vietnamese-RVC)

---

**Acknowledgments**

To all external collaborators for their special help in the following areas: Original RVC / Mainline (for making RVC), Vietnamese-RVC (for their up-to-date Fork used as a Base).

**Disclaimer**

By using Ultimate RVC Maker, you agree to comply with ethical and legal standards, respect intellectual property and privacy rights, avoid harmful or prohibited uses, and accept full responsibility for any outcomes, while Ultimate RVC Maker disclaims liability and reserves the right to amend these terms.

In [ ]:
# Install Ultimate RVC Maker

import os
from IPython.display import clear_output
print("👩🏻‍💻 INSTALLING...")

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

!git clone https://github.com/unchCrew/RVC-MAKER.git /kaggle/working/program
!pip install -r /kaggle/working/program/requirements.txt
!pip install pyngrok
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# Installation will take about 2 minutes to complete!
clear_output()
print("✅ Finished installing requirements!")

In [ ]:
# Run web UI

import os

%cd /kaggle/working/program

# Note: Kaggle does not support external sharing methods like gradio, localtunnel, or ngrok.
# The application will run locally within the Kaggle notebook environment.
# To access the web UI, you may need to run this notebook locally or use a different platform.

tensorboard = False  # Set to True to enable TensorBoard for checking overtraining

if tensorboard:
    %load_ext tensorboard
    %tensorboard --logdir ./assets/logs --port=6870

!python main/app/app.py

# Extra

Enjoy extra options that can make it easier for you to use Ultimate RVC Maker.

In [ ]:
# Backup to Kaggle Output
# Note: This replaces Google Drive backup with Kaggle's output directory.

import os
import shutil
import time

LOGS_FOLDER = "/kaggle/working/program/assets/logs/"
BACKUP_PATH = "/kaggle/working/RVCBackup"

if "autobackups" not in globals():
    autobackups = False

cooldown = 15  # Adjust the cooldown period (in seconds) as needed

def backup_files():
    print("\nStarting backup loop...")
    last_backup_timestamps_path = os.path.join(LOGS_FOLDER, "last_backup_timestamps.txt")
    fully_updated = False

    while True:
        try:
            updated_files = 0
            deleted_files = 0
            new_files = 0
            last_backup_timestamps = {}

            try:
                with open(last_backup_timestamps_path, "r") as f:
                    last_backup_timestamps = dict(line.strip().split(":") for line in f)
            except FileNotFoundError:
                pass

            for root, dirs, files in os.walk(LOGS_FOLDER):
                if "zips" in dirs:
                    dirs.remove("zips")
                if "mute" in dirs:
                    dirs.remove("mute")

                for filename in files:
                    if filename != "last_backup_timestamps.txt":
                        filepath = os.path.join(root, filename)
                        if os.path.isfile(filepath):
                            backup_filepath = os.path.join(BACKUP_PATH, os.path.relpath(filepath, LOGS_FOLDER))
                            backup_folderpath = os.path.dirname(backup_filepath)
                            if not os.path.exists(backup_folderpath):
                                os.makedirs(backup_folderpath)
                            last_backup_timestamp = last_backup_timestamps.get(filepath)
                            current_timestamp = os.path.getmtime(filepath)
                            if (last_backup_timestamp is None or float(last_backup_timestamp) < current_timestamp):
                                shutil.copy2(filepath, backup_filepath)
                                last_backup_timestamps[filepath] = str(current_timestamp)
                                if last_backup_timestamp is None:
                                    new_files += 1
                                else:
                                    updated_files += 1

            for filepath in list(last_backup_timestamps.keys()):
                if not os.path.exists(filepath):
                    backup_filepath = os.path.join(BACKUP_PATH, os.path.relpath(filepath, LOGS_FOLDER))
                    if os.path.exists(backup_filepath):
                        os.remove(backup_filepath)
                        deleted_files += 1
                    del last_backup_timestamps[filepath]

            if updated_files > 0 or deleted_files > 0 or new_files > 0:
                print(f"Backup Complete: {new_files} new, {updated_files} updated, {deleted_files} deleted.")
                fully_updated = False
            elif not fully_updated:
                print("Files are up to date.")
                fully_updated = True

            with open(last_backup_timestamps_path, "w") as f:
                for filepath, timestamp in last_backup_timestamps.items():
                    f.write(f"{filepath}:{timestamp}\n")

            time.sleep(cooldown if fully_updated else 0.1)

        except Exception as error:
            print(f"An error occurred during backup: {error}")

if autobackups:
    autobackups = False
    print("Autobackup Disabled")
else:
    autobackups = True
    print("Autobackup Enabled")

In [ ]:
# Upload model to Hugging Face

repo_hf = "NeoPy/TTS-G"  # Replace with your Hugging Face repository
pth = "/kaggle/working/program/assets/weights/TTS_100e_500s.pth"  # Path to model file
index = "/kaggle/working/program/assets/logs/TTS/added_IVF59_Flat_nprobe_1_TTS_v2.index"  # Path to index file
token = "hf_"  # Replace with your Hugging Face token (get from https://huggingface.co/settings/tokens)

import huggingface_hub
import zipfile
import os

def upload_model(repo, pth, index, token):
    readme = f"""
    # {repo}
    This is a model uploaded by RVC Maker, using [NeoDev](https://github.com/TheNeodev)'s script.
    """
    repo_name = repo.split('/')[1]
    with zipfile.ZipFile(f'{repo_name}.zip', 'w') as zipf:
        zipf.write(pth, os.path.basename(pth))
        zipf.write(index, os.path.basename(index))
        zipf.writestr('README.md', readme)

    huggingface_hub.HfApi().create_repo(repo_id=repo, token=token, exist_ok=True)
    huggingface_hub.HfApi().upload_file(token=token, path_or_fileobj=f'{repo_name}.zip', repo_id=repo, path_in_repo=f'{repo_name}.zip')
    os.remove(f'{repo_name}.zip')
    return "Model uploaded successfully"

upload_model(repo_hf, pth, index, token)